In [39]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from datetime import datetime, date

In [4]:
r = requests.get('https://statusinvest.com.br/acoes/itub4', headers={
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'
})
assert r.status_code == 200, f'error fetching: {r.text}'

In [5]:
bs = BeautifulSoup(r.content)

In [17]:
divs_data = bs.find_all('input', {'id': 'results'})[0]

In [24]:
df = pd.DataFrame(json.loads(divs_data['value']))
df = df[['ed', 'et', 'v']].rename({'ed': 'com_date', 'et': 'div_type', 'v': 'value'}, axis=1)
df['com_date'] = df['com_date'].map(lambda d: datetime.strptime(d, '%d/%m/%Y').date())
df = df.set_index('com_date').sort_index()

In [42]:
divs_2021 = df.loc[date(2021, 1, 1):date(2021, 12, 31)]

In [44]:
divs_2021.to_parquet('data/itub_divs_2021.parquet')